In [ ]:
!pip install torch transformers spacy keybert PyPDF2
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 125.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import spacy
from keybert import KeyBERT
from PyPDF2 import PdfReader

In [ ]:
device = 0 if torch.cuda.is_available() else -1
gpu_status = "GPU: " + torch.cuda.get_device_name(0) if device == 0 else "CPU"
print(f" Running on {gpu_status}")

 Running on GPU: Tesla T4


In [ ]:
nlp = spacy.load("en_core_web_sm")
kw_model = KeyBERT()

In [ ]:
from google.colab import drive

In [ ]:
!unzip /content/drive/MyDrive/multitask_fda_extractor.zip -d /content/drive/MyDrive/multitask_fda_extractor

Archive:  /content/drive/MyDrive/multitask_fda_extractor.zip
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/special_tokens_map.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/labels.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/tokenizer.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
MODEL_PATH = "/content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/"

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
model = BertForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=6)  # 6 heads
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/ and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
HEADS = ["Hardware", "Software Components", "AI Models", "Data Pipelines", "User Interface", "Integration"]
# Helper: run multitask predictions
def predict_heads(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits.squeeze()
    probs = torch.sigmoid(logits).cpu().numpy()
    return {HEADS[i]: float(probs[i]) for i in range(len(HEADS))}

# Helper: extract keywords from text
def extract_keywords(text, top_n=5):
    doc = nlp(text)
    chunks = list(set(chunk.text.lower() for chunk in doc.noun_chunks if len(chunk.text) > 3))
    if chunks:
        return chunks[:top_n]
    keybert_kw = kw_model.extract_keywords(text, top_n=top_n)
    return [kw for kw, _ in keybert_kw]

# Split PDF into text chunks
def extract_chunks_from_pdf(pdf_path, chunk_size=300):
    reader = PdfReader(pdf_path)
    full_text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
    words = full_text.split()
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    print(f"Extracted {len(chunks)} chunks from PDF: {chunks}")
    return chunks

# Main pipeline: classify chunks + extract keywords
def analyze_text_chunks(chunks, threshold=0.5):
    results = {head: [] for head in HEADS}
    for chunk in chunks:
        head_probs = predict_heads(chunk)
        for head, prob in head_probs.items():
            if prob >= threshold:
                kws = extract_keywords(chunk)
                results[head].extend(kws)
    for head in results:
        results[head] = list(set(results[head]))
    return results

In [ ]:
from google.colab import files
uploaded = files.upload()
PDF_PATH = list(uploaded.keys())[0]
chunks = extract_chunks_from_pdf(PDF_PATH)
output = analyze_text_chunks(chunks)
print(output)

Saving K250236.pdf to K250236 (4).pdf
Extracted 11 chunks from PDF: ['U.S. Food & Drug Administration 10903 New Hampshire Avenue D o c I D # 0 4 0 1 7 . 0 7.05 Silver Spring, MD 20993 www.fda.gov May 30, 2025 Hyperfine, Inc. Christine Kupchick Sr. Manager, Global Regulatory 351 New Whitfield St Guilford, Connecticut 06437 Re: K250236 Trade/Device Name: Swoop® Portable MR Imaging® System (V2) Regulation Number: 21 CFR 892.1000 Regulation Name: Magnetic Resonance Diagnostic Device Regulatory Class: Class II Product Code: LNH, MOS Dated: May 7, 2025 Received: May 7, 2025 Dear Christine Kupchick: We have reviewed your section 510(k) premarket notification of intent to market the device referenced above and have determined the device is substantially equivalent (for the indications for use stated in the enclosure) to legally marketed predicate devic es marketed in interstate commerce prior to May 28, 1976, the enactment date of the Medical Device Amendments, or to devices that have been rec